In [57]:
# get all orders
import urllib.parse
import hashlib
import hmac
import base64
import requests
import time
import csv
import json

api_key = 'ZxOQ7t6879VgwLQzhxIixxB39iIAUX2355YMjFu1yQzDgUmpquOq3Gyb3XESVJBs'
secret_key = 'tjuC6WfLgKNabCbC21KGJq0uF3XjEzvuXVBU1rPuW7FIrbra4IfOfw4pqILLHT5i'

api_url = "https://api.binance.us"

# get binanceus signature
def get_binanceus_signature(data, secret):
    postdata = urllib.parse.urlencode(data)
    message = postdata.encode()
    byte_key = bytes(secret, 'UTF-8')
    mac = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return mac

# Attaches auth headers and returns results of a POST request
def binanceus_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['X-MBX-APIKEY'] = api_key
    signature = get_binanceus_signature(data, api_sec)
    params={
        **data,
        "signature": signature,
        }
    req = requests.get((api_url + uri_path), params=params, headers=headers)
    return req.text

uri_path = "/api/v3/allOrders"
data = {
    "timestamp": int(round(time.time() * 1000)),
    "symbol": "BTCUSDT"
}

result = binanceus_request(uri_path, data, api_key, secret_key)

result_obj = json.loads(result)

# print result_obj with 4 indentations
# print(json.dumps(result_obj, indent=4))

# Open the file in write mode ('w') and create a CSV writer
with open('trade_history.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write a header row
    writer.writerow(['Quantity', 'Price', 'Side'])

    # Iterate over the result and write each open position to the file
    for position in result_obj:

        if position['status'] == 'FILLED':
            if position['price'] == '0.00000000':
                price = float(position['cummulativeQuoteQty'])/float(position['executedQty'])
                quantity = float(position['executedQty'])
                side = position['side']
                time_trade = position['time']
                writer.writerow([quantity, price, side, time_trade])
            else: 
                price = float(position['price'])
                quantity = float(position['executedQty'])
                side = position['side']
                time_trade = position['time']
                writer.writerow([quantity, price, side, time_trade])

print("trade history saved to trade_history.csv")

################################################

# Read trade history from file
with open('trade_history.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row
    positions = list(reader)

# only work with trades that have been completed in a time period
# get current time
current_time = int(round(time.time() * 1000))
# get 12:30am of July 6th, 2023
time_run = 1688628630000

# get time 24 hours ago
time_24_hours_ago = current_time - 86400000
# get 1.5 hours ago
time_1_5_hours_ago = current_time - 5400000
# get all trades that have been completed in the last 24 hours
completed_trades = []
for position in positions:
    quantity = float(position[0])
    price = float(position[1])
    side = position[2]
    time_trade = int(position[3])
    if time_trade > time_run:
        completed_trades.append(position)

positions = completed_trades

# calculate number of trades
num_trades = len(positions)
print("Number of filled trades: {}".format(num_trades))
# Compute average purchase price
total_purchased = 0
for position in positions:
    quantity = float(position[0])
    price = float(position[1])
    side = position[2]
    if side == 'BUY':
        total_purchased += quantity * price
avg_purchase_price = total_purchased / num_trades
print("Average purchase price: ${}".format(avg_purchase_price))

# Compute average sale price
total_sold = 0
for position in positions:
    quantity = float(position[0])
    price = float(position[1])
    side = position[2]
    if side == 'SELL':
        total_sold += quantity * price
avg_sale_price = total_sold / num_trades
print("Average sale price: ${}".format(avg_sale_price))

# Compute total profit/loss
total_pnl = 0
for position in positions:
    quantity = float(position[0])
    price = float(position[1])
    side = position[2]
    if side == 'BUY':
        total_pnl -= quantity * price
    else:
        total_pnl += quantity * price

print("Total PnL: ${}".format(total_pnl))


# total PnL you miss if you close all open positions
total_risk_open_positions = 1.66 # read from another file
num_open_positions = 34 # read from another file

# real number of full double-trades (SELL-BUY or BUY-SELL)
num_filled_trades = num_trades
num_double_trades = (num_filled_trades - num_open_positions)//2


print("Number of double trades: {}".format(num_double_trades))
real_pnl = total_pnl - total_risk_open_positions
print("Real PnL: ${}".format(real_pnl))

trade history saved to trade_history.csv
Number of filled trades: 104
Average purchase price: $1.2846255288461539
Average sale price: $1.8194105750000005
Total PnL: $55.617644799999994
Number of double trades: 35
Real PnL: $53.9576448
